# EU4 AI Training with Gemma-3-270M

Train LoRA adapters using Google's Gemma-3 270M model - a compact but capable model trained on 6T tokens.

**Why Gemma-3-270M?**
- 270M params (similar to SmolLM2-360M but newer architecture)
- Trained on 6 trillion tokens (more than SmolLM2)
- 32K context window
- Strong reasoning despite small size

**One-time setup:**
1. Accept Gemma license at https://huggingface.co/google/gemma-3-270m
2. Create HuggingFace access token at https://huggingface.co/settings/tokens
3. In Colab sidebar: 🔑 Secrets → Add `HF_TOKEN` with your token
4. Create a GitHub PAT at https://github.com/settings/tokens (classic, `repo` scope)
5. Add secret: `GITHUB_TOKEN` with your PAT
6. Create folder in Google Drive (e.g., `eu4_training/`)

**Per-session:**
1. Upload `.cpb.zip` training data to Drive
2. Run all cells in order

In [ ]:
# Check GPU - Gemma-3-270M fits comfortably on T4/L4
# If you see "No GPU", go to Runtime → Change runtime type → GPU
import torch

if not torch.cuda.is_available():
    print("⚠️  NO GPU DETECTED!")
    print("Go to: Runtime → Change runtime type → Hardware accelerator → GPU")
    print("Then re-run this cell.")
else:
    !nvidia-smi
    print(f"✓ GPU available: {torch.cuda.get_device_name(0)}")

In [ ]:
# Mount Google Drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# Authenticate with HuggingFace for Gemma access
from google.colab import userdata
from huggingface_hub import login

hf_token = userdata.get("HF_TOKEN")
login(token=hf_token)
print("✓ Authenticated with HuggingFace")

In [ ]:
# Clone repo from GitHub
import os
from google.colab import userdata

token = userdata.get("GITHUB_TOKEN")

if os.path.exists("eu4rs"):
    !rm -rf eu4rs

!git clone --depth 1 https://{token}@github.com/atvrager/eu4rs.git

print("✓ Cloned eu4rs from GitHub")

In [ ]:
# Install dependencies
import os

os.environ["WANDB_DISABLED"] = "true"

# Flash attention for Gemma efficiency (optional but recommended)
!pip install -q transformers peft trl datasets pycapnp safetensors
!pip install -q flash-attn --no-build-isolation 2>/dev/null || echo "Flash attention not available, using default"

print("✓ Dependencies installed")

In [ ]:
# Configuration - EDIT THESE
# ===========================

# Path to your training data in Google Drive
DATA_PATH = "/content/drive/MyDrive/eu4_training/run_10yr_1.cpb.zip"

# Where to save the trained adapter
OUTPUT_DIR = "/content/drive/MyDrive/eu4_training/adapters/gemma3-270m-run1"

# Model: Gemma-3-270M (base model for fine-tuning)
BASE_MODEL = "google/gemma-3-270m"

# Training settings - MEMORY OPTIMIZED for T4 (16GB)
# If OOM: reduce BATCH_SIZE to 2, or use gradient checkpointing
MAX_STEPS = 10000
BATCH_SIZE = 4  # Reduced from 8 to avoid OOM on T4
GRAD_ACCUM = 4  # Increased to maintain effective batch size of 16
SAVE_STEPS = 2500

# Repo paths
REPO_DIR = "/content/eu4rs"
SCRIPTS_DIR = f"{REPO_DIR}/scripts"

# Verify paths
import os

assert os.path.exists(DATA_PATH), f"Data not found: {DATA_PATH}"
assert os.path.exists(SCRIPTS_DIR), f"Repo not cloned: {SCRIPTS_DIR}"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✓ Data: {DATA_PATH}")
print(f"✓ Model: {BASE_MODEL}")
print(f"✓ Output: {OUTPUT_DIR}")
print(f"✓ Effective batch size: {BATCH_SIZE * GRAD_ACCUM}")

In [ ]:
# Verify data format
import sys

sys.path.insert(0, SCRIPTS_DIR)

from load_training_data import iter_batches_raw

print("Checking first batch...")
for batch in iter_batches_raw(DATA_PATH):
    print(f"✓ Loaded batch with {len(batch.samples)} samples")
    sample = batch.samples[0]
    print(f"  First sample: {sample.country} @ tick {sample.tick}")
    break
print("✓ Data format verified")

In [ ]:
# Run training!
!cd {SCRIPTS_DIR} && python train_ai.py \
    --data "{DATA_PATH}" \
    --base-model "{BASE_MODEL}" \
    --output "{OUTPUT_DIR}" \
    --max-steps {MAX_STEPS} \
    --save-steps {SAVE_STEPS} \
    --batch-size {BATCH_SIZE} \
    --grad-accum {GRAD_ACCUM} \
    --prefetch 1000

In [ ]:
# Verify output
import os

files = os.listdir(OUTPUT_DIR)
print(f"Adapter files in {OUTPUT_DIR}:")
for f in sorted(files):
    path = os.path.join(OUTPUT_DIR, f)
    if os.path.isfile(path):
        size = os.path.getsize(path)
        print(f"  {f}: {size / 1024:.1f} KB")
    else:
        print(f"  {f}/ (checkpoint)")

## Resume Training

If Colab disconnects, resume from the last checkpoint:

In [ ]:
# Find latest checkpoint
import os
import re

checkpoints = [d for d in os.listdir(OUTPUT_DIR) if d.startswith("checkpoint-")]
if checkpoints:
    latest = max(checkpoints, key=lambda x: int(re.search(r"\d+", x).group()))
    RESUME_FROM = os.path.join(OUTPUT_DIR, latest)
    print(f"Latest checkpoint: {RESUME_FROM}")
else:
    print("No checkpoints found")
    RESUME_FROM = None

In [ ]:
# Resume training
if RESUME_FROM:
    !cd {SCRIPTS_DIR} && python train_ai.py \
        --data "{DATA_PATH}" \
        --base-model "{BASE_MODEL}" \
        --output "{OUTPUT_DIR}" \
        --max-steps {MAX_STEPS} \
        --save-steps {SAVE_STEPS} \
        --batch-size {BATCH_SIZE} \
        --grad-accum {GRAD_ACCUM} \
        --prefetch 1000 \
        --resume-from "{RESUME_FROM}"
else:
    print("No checkpoint to resume from. Run initial training first.")

## Compare with SmolLM2

| Aspect | SmolLM2-360M | Gemma-3-270M |
|--------|--------------|---------------|
| Parameters | 360M | 270M |
| Training data | 4T tokens | 6T tokens |
| Context | 8K | 32K |
| Architecture | LLaMA-like | Gemma-3 |
| License | Apache 2.0 | Gemma |

**Hypothesis:** Gemma-3's better training data efficiency may produce stronger EU4 decisions despite fewer parameters.